<h1 align="center"><font color="yellow">PGVector</font></h1>

<font color="yellow">Data Scientist.: Dr.Eddy Giusepe Chirinos Isidro</font>

In [24]:
from dotenv import load_dotenv
from langchain.embeddings import OpenAIEmbeddings
from psycopg.conninfo import make_conninfo
import psycopg

import os
import openai
from dotenv import find_dotenv, load_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key  = os.getenv('OPENAI_API_KEY')


embeddings = OpenAIEmbeddings()

connectionString = make_conninfo(
    host=os.getenv('DB_HOST'),
    port=os.getenv('DB_PORT'),
    dbname=os.getenv('DB_NAME'),
    user=os.getenv('DB_USER'),
    password=os.getenv('DB_PASSWORD')
)

In [25]:
# Carregar documentos de amostra no banco de dados:

directory = './data'

# Percorra todos os arquivos no diretório, crie incorporações, salve-os no banco de dados:
for file_name in os.listdir(directory):
  file_path = os.path.join(directory, file_name)
  with open(file_path, 'r') as file:
      content = file.read()
      embedding_results = embeddings.embed_query(content)
      with psycopg.connect(connectionString) as conn:
        insert_query = "INSERT INTO documents (file_name, content, embedding) VALUES (%s, %s, %s)"
        insert_data = (file_name, content, embedding_results)
        conn.execute(insert_query, insert_data)
        conn.commit()

# NOTA: Você precisa criar o TABELA, lá no DB postgres, assim:

'''
CREATE TABLE documents (
    id serial PRIMARY KEY,
    file_name varchar,
    content text,
    embedding double precision[]
);
'''        


'\nCREATE TABLE documents (\n    id serial PRIMARY KEY,\n    file_name varchar,\n    content text,\n    embedding double precision[]\n);\n'

In [74]:
search_text = "Como se chamava a tartaruga?"


# search do texto Embedding:
embedding_results = embeddings.embed_query(search_text)

len(embedding_results)

1536

In [ ]:
# Encontre documentos com base na SIMILARIDADE de COSSENO com o texto de pesquisa Embedding:
with psycopg.connect(connectionString) as conn:
   select_query = "SELECT * FROM documents"
   query_data = (f'{embedding_results}', 0.8, 10)
   results = conn.execute(select_query, query_data).fetchall()
   for result in results:
     print(result) 

